In [1]:
import pypsa 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from custom_constraints import set_operational_limits

In [2]:
model_file = pd.ExcelFile("../data/model_file.xlsx")
model_setup = (
    pd.read_excel(
        model_file, 
        sheet_name="model_setup",
        index_col=[0])
        .loc["grid-expansion"]
)


In [12]:
#n0 = pypsa.Network('../networks/elec_grid-2040_30-supply_redz.nc')
n0 = pypsa.Network('../networks/pre_grid-expansion_11-supply_redz_lcopt_LC-3000SEG.nc')

INFO:pypsa.io:Imported network pre_grid-expansion_11-supply_redz_lcopt_LC-3000SEG.nc has buses, carriers, generators, global_constraints, lines, loads, storage_units


In [13]:
n0.optimize.create_model(multi_investment_periods=True)
set_operational_limits(n0, model_file, model_setup)

In [14]:
n0.model.to_netcdf("test_network.nc")

ValueError: cannot re-index or align objects with conflicting indexes found for the following coordinates: 'period' (30 conflicting indexes), 'timestep' (30 conflicting indexes)
Conflicting indexes may occur when
- they relate to different sets of coordinate and/or dimension names
- they don't have the same type
- they may be used to reindex data along common dimensions

In [5]:
#n0.optimize.create_model(multi_investment_periods=True)
n0.model.constraints.remove("Kirchhoff-Voltage-Law")
n0.optimize.solve_model(solver_name='xpress', solver_options = {"lpflags":4, "crossover":0})

INFO:linopy.model: Solve problem using Xpress solver
INFO:linopy.model:Solver options:
 - lpflags: 4
 - crossover: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 9/9 [00:01<00:00,  4.75it/s]
INFO:linopy.io: Writing time: 18.76s


Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/home/pklein/anaconda3/envs/pypsa_github/lib/python3.11/site-packages/xpress/license/community-xpauth.xpr')
 
Reading Problem linopy-problem-z6jf064h
Problem Statistics
     8784000 (      0 spare) rows
     2979206 (      0 spare) structural columns
    15196501 (      0 spare) non-zero elements
MIP Entity Statistics
           0 entities        0 sets        0 set members
FICO Xpress v9.1.3, Hyper, solve started 13:46:25, Nov 9, 2023
Heap usage: 2402MB (peak 3915MB, 102MB system)
Minimizing LP linopy-problem-z6jf064h using up to 20 threads and up to 117GB memory, with these control settings:
OUTPUTLOG = 1
CROSSOVER = 0
LPFLAGS = 4
Original problem has:
   8784000 rows      2979206 cols     15196

Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: lp_infeas



('warning', 'infeasible')

In [6]:
n0.export_to_netcdf("test_network.nc")

INFO:pypsa.io:Exported network test_network.nc has storage_units, loads, lines, generators, global_constraints, carriers, buses


<xarray.Dataset>
Dimensions:                               (snapshots: 12000,
                                           investment_periods: 4,
                                           storage_units_i: 42, loads_i: 11,
                                           loads_t_p_set_i: 11, lines_i: 26,
                                           generators_i: 206,
                                           generators_t_p_min_pu_i: 42,
                                           generators_t_p_max_pu_i: 179,
                                           generators_t_p_nom_pu_i: 64,
                                           global_constraints_i: 8,
                                           carriers_i: 12, buses_i: 11)
Coordinates: (12/13)
  * snapshots                             (snapshots) int64 0 1 ... 11998 11999
  * investment_periods                    (investment_periods) int64 2025 ......
  * storage_units_i                       (storage_units_i) object 'Drakensbe...
  * loads_i                               (loads_i) object 'Pelly' ... 'Easte...
  * loads_t_p_set_i                       (loads_t_p_set_i) object 'Pelly' .....
  * lines_i                               (lines_i) object '0' '1' ... '24' '25'
    ...                                    ...
  * generators_t_p_min_pu_i               (generators_t_p_min_pu_i) object 'L...
  * generators_t_p_max_pu_i               (generators_t_p_max_pu_i) object 'M...
  * generators_t_p_nom_pu_i               (generators_t_p_nom_pu_i) object 'M...
  * global_constraints_i                  (global_constraints_i) object 'glob...
  * carriers_i                            (carriers_i) object 'coal' ... 'phs'
  * buses_i                               (buses_i) object 'Pelly' ... 'Easte...
Data variables: (12/70)
    snapshots_period                      (snapshots) int32 2025 2025 ... 2040
    snapshots_timestep                    (snapshots) datetime64[ns] 2025-01-...
    snapshots_objective                   (snapshots) float64 2.0 4.0 ... 2.0
    snapshots_generators                  (snapshots) float64 2.0 4.0 ... 2.0
    snapshots_stores                      (snapshots) float64 2.0 4.0 ... 2.0
    investment_periods_objective          (investment_periods) float64 4.297 ...
    ...                                    ...
    buses_nom_max_nuclear_2025            (buses_i) float64 nan nan ... nan 0.0
    buses_nom_min_wind_2025               (buses_i) float64 nan 5e+03 ... nan
    buses_nom_min_battery_2030            (buses_i) float64 nan 500.0 ... nan
    buses_nom_min_battery_2035            (buses_i) float64 nan 1e+03 ... nan
    buses_control                         (buses_i) object 'PQ' 'PQ' ... 'PQ'
    buses_generator                       (buses_i) object '' '' '' ... '' '' ''
Attributes:
    network__linearized_uc:  0
    network__multi_invest:   1
    network_multi_invest:    1
    network_name:            PyPSA-ZA
    network_pypsa_version:   0.25.2
    network_srid:            4326
    meta:                    {"_crs": "GEOGCRS[\"WGS 84\",ENSEMBLE[\"World Ge...

In [ ]:
n0.model.constraints["min-ocgt_gas-month-2025"].sel(month=1)

Constraint `min-ocgt_gas-month-2025`
------------------------------------
+2 Generator-p[(2025, Timestamp('2025-01-01 00:00:00')), Gauteng-ocgt_gas_Sasol] + 4 Generator-p[(2025, Timestamp('2025-01-01 02:00:00')), Gauteng-ocgt_gas_Sasol] + 2 Generator-p[(2025, Timestamp('2025-01-01 06:00:00')), Gauteng-ocgt_gas_Sasol] ... +2 Generator-p[(2025, Timestamp('2025-01-31 16:00:00')), Western Cape-ocgt_gas_eskom] + 4 Generator-p[(2025, Timestamp('2025-01-31 18:00:00')), Western Cape-ocgt_gas_eskom] + 2 Generator-p[(2025, Timestamp('2025-01-31 22:00:00')), Western Cape-ocgt_gas_eskom] ≥ 285324.0

In [ ]:
n0.model.variables.get_label_position(57528)

('Generator-p',
 {'snapshot': (2025, Timestamp('2025-02-01 00:00:00')),
  'Generator': 'Gauteng-ocgt_gas_Sasol'})

In [ ]:
n0.generators.p_nom_opt.groupby(n0.generators.carrier).sum()

carrier
biomass         0.0
ccgt_gas        0.0
coal            0.0
hydro           0.0
hydro_import    0.0
nuclear         0.0
ocgt_diesel     0.0
ocgt_gas        0.0
solar_csp       0.0
solar_pv        0.0
wind            0.0
Name: p_nom_opt, dtype: float64

In [ ]:
n0.storage_units.query("carrier == 'phs'").p_nom_opt

StorageUnit
Drakensberg               0.0
Ingula                    0.0
Palmiet                   0.0
Steenbras                 0.0
Western Cape-phs-2035     0.0
Western Cape-phs-2040     0.0
Gauteng-phs-2035          0.0
Gauteng-phs-2040          0.0
KwaZulu Natal-phs-2035    0.0
KwaZulu Natal-phs-2040    0.0
Name: p_nom_opt, dtype: float64

In [ ]:
n0.generators_t.p[n0.generators[n0.generators.carrier=="wind"].index].sum()/1e6

KeyError: "None of [Index(['Eastern Cape-wind_BW1', 'Hydra Cluster-wind_BW1',\n       'Northern Cape-wind_BW1', 'Western Cape-wind_BW1',\n       'Eastern Cape-wind_BW2', 'Hydra Cluster-wind_BW2',\n       'Western Cape-wind_BW2', 'Eastern Cape-wind_BW3',\n       'Northern Cape-wind_BW3', 'Western Cape-wind_BW3',\n       'Eastern Cape-wind_BW4', 'Hydra Cluster-wind_BW4',\n       'KwaZulu Natal-wind_BW4', 'Northern Cape-wind_BW4',\n       'Western Cape-wind_BW4', 'Eastern Cape-wind_BW5',\n       'Hydra Cluster-wind_BW5', 'Western Cape-wind_BW5',\n       'Eastern Cape-wind_BW6', 'Hydra Cluster-wind_BW6',\n       'Western Cape-wind_BW6', 'Western Cape-wind-2025',\n       'Western Cape-wind-2030', 'Western Cape-wind-2035',\n       'Western Cape-wind-2040', 'Northern Cape-wind-2025',\n       'Northern Cape-wind-2030', 'Northern Cape-wind-2035',\n       'Northern Cape-wind-2040', 'Eastern Cape-wind-2025',\n       'Eastern Cape-wind-2030', 'Eastern Cape-wind-2035',\n       'Eastern Cape-wind-20

In [ ]:
n0.generators_t.p[n0.generators[n0.generators.carrier=="coal"].index].sum().sum()/1e6

In [ ]:
n0.generators_t.p[n0.generators[n0.generators.carrier=="solar_pv"].index].sum().sum()/1e6

In [ ]:
n0.loads_t.p_set.sum().sum()/1e6

In [ ]:
n0.storage_units.query("carrier=='phs'")

In [ ]:
n0.statistics()

In [ ]:
n0.loads_t.p_set.sum(axis=1).plot()

In [ ]:
n0.generators_t.p_max_pu.mean()

In [ ]:
n0.generators.query("p_nom_extendable == False")

In [ ]:
n0.generators_t.p.mean()

In [ ]:
#n0.buses.loc["Northern Cape", "nom_max_wind"] = 2000
n0.buses.loc["Hydra Cluster", "nom_max_solar_pv"] = 1000
n0.buses.loc["Gauteng", "nom_min_nuclear"] = 2200

In [ ]:
#n0.export_to_netcdf('../networks/solved_elec_grid-expansion_11-supply_redz.nc')

In [ ]:
model_setup = "base"
model_file = pd.ExcelFile('../data/model_file.xlsx')

ext_max_build = (
    pd.read_excel(
        model_file, 
        sheet_name='extendable_max_build',
        index_col=[0,1,2,3])
).loc[model_setup]
ext_max_build.replace("unc", np.inf, inplace=True)

# global max build


# bus-carrier max_build








ext_min_build = (
    pd.read_excel(
        model_file, 
        sheet_name='extendable_min_build',
        index_col=[0,1,2])
).loc[model_setup]
ext_min_build.replace(0, np.nan, inplace=True)

# build_constraints = (pd.read_excel('../data/model_file.xlsx', 
#                             sheet_name='new_build_limits',
#                             index_col=[0,1,2])).loc[model_setup['new_build_limits']]

# max_build = build_constraints.loc['max_installed_limit'].fillna(100000)
# min_build = build_constraints.loc['min_installed_limit']
         


In [ ]:
ext_max_build.loc[("11-supply","Western Cape")]

In [ ]:
n0.generators.p_nom_opt.groupby(n0.generators.carrier).sum()

In [ ]:
n0.storage_units.p_nom_opt.groupby(n0.storage_units.carrier).sum()

In [ ]:
n0.statistics()